In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('/Users/flatironschool/Documents/nyc-ds-010620/Projects/baseball_HR_linear_regression/Loading_Dataset/cleaned_data.csv', index_col=0)

In [13]:
df

,AS__2,AS__3,Age,HR/PA,IBB/PA,OBP,OPS,RBI/PA,SF/PA,SH/PA,...,Tm__TOR,Tm__TOT,Tm__WSN,avg_hit_angle,avg_hit_speed,avg_hr_distance,brl_pa,est_slg,est_woba,ev95percent
0,1.0,0.0,25.0,0.033473,0.011611,0.353,0.812,0.095791,0.008708,0.004354,...,NaN,NaN,NaN,10.5,85.8,386.0,2.6,0.390,0.305,28.8
1,1.0,0.0,28.0,0.010264,0.001511,0.293,0.614,0.070997,0.007553,0.016616,...,NaN,NaN,NaN,8.3,83.5,395.0,0.8,0.337,0.287,16.1
2,1.0,0.0,30.0,0.021672,0.002928,0.314,0.709,0.112738,0.014641,0.002928,...,NaN,NaN,NaN,9.5,89.4,398.0,1.9,0.377,0.311,36.0
3,0.0,1.0,33.0,0.041237,0.000000,0.342,0.770,0.108148,0.007407,0.000000,...,NaN,NaN,NaN,17.4,85.6,378.0,2.4,0.391,0.315,26.2
4,0.0,1.0,27.0,0.005333,0.000000,0.342,0.719,0.070978,0.004732,0.007886,...,NaN,NaN,NaN,3.9,82.1,367.0,0.0,0.355,0.307,14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(df, df['HR/PA'],
                                                    test_size=0.1,
                                                    random_state=0) # we are setting the seed here
X_train.shape, X_test.shape

((1548, 48), (172, 48))